In [1]:
from Datos import Datos
from EstrategiaParticionado import *
from Clasificador import *

In [2]:
k = 2
p = 0.2
nEjecuciones = 10

dataset = Datos("german.data")

#Cambio estrategia particionado comentar

In [3]:
print(dataset.diccionario)

[{'A11': 0, 'A12': 1, 'A13': 2, 'A14': 3}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}]


In [4]:

estrategia_simple = ValidacionSimple(p, nEjecuciones)
particiones_simple = estrategia_simple.creaParticiones(10)

for particion in particiones_simple:
    datos_test = dataset.extraeDatos(particion.indicesTest)
    datos_train = dataset.extraeDatos(particion.indicesTrain)
    print("Test:", str(datos_test))
    print("Train:", str(datos_train))
    print()
    

Test: [array(['A11', 6, 'A34', 'A43', 1169, 'A65', 'A75', 4, 'A93', 'A101', 4,
       'A121', 67, 'A143', 'A152', 2, 'A173', 1, 'A192', 'A201', 1],
      dtype=object), array(['A11', 42, 'A32', 'A42', 7882, 'A61', 'A74', 2, 'A93', 'A103', 4,
       'A122', 45, 'A143', 'A153', 1, 'A173', 2, 'A191', 'A201', 1],
      dtype=object)]
Train: [array(['A12', 30, 'A34', 'A40', 5234, 'A61', 'A71', 4, 'A94', 'A101', 2,
       'A123', 28, 'A143', 'A152', 2, 'A174', 1, 'A191', 'A201', 2],
      dtype=object), array(['A12', 36, 'A32', 'A41', 6948, 'A61', 'A73', 2, 'A93', 'A101', 2,
       'A123', 35, 'A143', 'A151', 1, 'A174', 1, 'A192', 'A201', 1],
      dtype=object), array(['A14', 12, 'A34', 'A46', 2096, 'A61', 'A74', 2, 'A93', 'A101', 3,
       'A121', 49, 'A143', 'A152', 1, 'A172', 2, 'A191', 'A201', 1],
      dtype=object), array(['A14', 12, 'A32', 'A43', 3059, 'A64', 'A74', 2, 'A91', 'A101', 4,
       'A121', 61, 'A143', 'A152', 1, 'A172', 1, 'A191', 'A201', 1],
      dtype=object), array(['

In [5]:
estrategia_cruzada = ValidacionCruzada(3)
particiones_cruzada = estrategia_cruzada.creaParticiones(10)

for particion in particiones_cruzada:
    """
    datos_test = dataset.extraeDatos(particion.indicesTest)
    datos_train = dataset.extraeDatos(particion.indicesTrain)
    print("Test:", str(datos_test))
    print("Train:", str(datos_train))
    print()
    """
    print(particion)
    

Test: [3, 4, 8]
Train: [1, 7, 2, 0, 6, 5, 9]
Test: [1, 7, 2]
Train: [3, 4, 8, 0, 6, 5, 9]
Test: [0, 6, 5]
Train: [3, 4, 8, 1, 7, 2, 9]


In [6]:
a = ValidacionSimple(0.1, 1)
b = a.creaParticiones(len(dataset.datos))

datos_prueba = dataset.extraeDatos(b[0].indicesTrain)
datos_test = dataset.extraeDatos(b[0].indicesTest)


In [7]:
c = ClasificadorNaiveBayes()
en = c.entrenamiento(datos_prueba, [], [])

[['A11', 'A12', 'A13', 'A14'], [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18, 20, 21, 22, 24, 26, 27, 28, 30, 33, 36, 39, 40, 42, 45, 47, 48, 54, 60, 72], ['A30', 'A31', 'A32', 'A33', 'A34'], ['A40', 'A41', 'A410', 'A42', 'A43', 'A44', 'A45', 'A46', 'A48', 'A49'], [250, 276, 338, 339, 343, 362, 368, 385, 392, 409, 426, 428, 433, 448, 454, 458, 484, 522, 571, 585, 590, 601, 609, 618, 625, 639, 640, 654, 660, 662, 666, 672, 674, 682, 683, 684, 685, 691, 697, 700, 701, 707, 709, 717, 719, 727, 730, 731, 745, 750, 753, 754, 759, 760, 763, 766, 776, 781, 783, 790, 797, 802, 804, 806, 841, 846, 866, 874, 882, 884, 886, 894, 900, 902, 907, 909, 915, 918, 926, 929, 930, 931, 932, 936, 937, 939, 947, 950, 951, 958, 959, 960, 975, 976, 983, 996, 999, 1007, 1024, 1028, 1037, 1038, 1042, 1047, 1048, 1049, 1050, 1053, 1055, 1056, 1068, 1076, 1082, 1098, 1101, 1103, 1107, 1108, 1113, 1123, 1126, 1131, 1136, 1138, 1149, 1154, 1155, 1158, 1163, 1164, 1169, 1175, 1185, 1188, 1193, 1198, 1199, 1200,

In [26]:
r = c.clasifica(datos_test, [], [])

cont = 0
for i in range(len(datos_test)):
    if datos_test[i][-1] == r[i]:
        cont += 1
    print(datos_test[i],"|", r[i])

print(cont, len(datos_test), cont/len(datos_test))

['A14' 18 'A30' 'A49' 4165 'A61' 'A73' 2 'A93' 'A101' 2 'A123' 36 'A142'
 'A152' 2 'A173' 2 'A191' 'A201' 2] | error
['A14' 21 'A33' 'A41' 2993 'A61' 'A73' 3 'A93' 'A101' 2 'A121' 28 'A142'
 'A152' 2 'A172' 1 'A191' 'A201' 1] | error
['A11' 24 'A32' 'A42' 2359 'A62' 'A71' 1 'A91' 'A101' 1 'A122' 33 'A143'
 'A152' 1 'A173' 1 'A191' 'A201' 2] | error
['A12' 9 'A32' 'A43' 1670 'A61' 'A72' 4 'A92' 'A101' 2 'A123' 22 'A143'
 'A152' 1 'A173' 1 'A192' 'A201' 2] | error
['A14' 36 'A34' 'A49' 6304 'A65' 'A75' 4 'A93' 'A101' 4 'A121' 36 'A143'
 'A152' 2 'A173' 1 'A191' 'A201' 1] | error
['A14' 36 'A32' 'A43' 3835 'A65' 'A75' 2 'A92' 'A101' 4 'A121' 45 'A143'
 'A152' 1 'A172' 1 'A192' 'A201' 1] | error
['A11' 12 'A32' 'A43' 709 'A61' 'A75' 4 'A93' 'A101' 4 'A121' 57 'A142'
 'A152' 1 'A172' 1 'A191' 'A201' 2] | 1
['A11' 18 'A30' 'A42' 3114 'A61' 'A72' 1 'A92' 'A101' 4 'A122' 26 'A143'
 'A151' 1 'A173' 1 'A191' 'A201' 2] | error
['A11' 24 'A34' 'A49' 1382 'A62' 'A74' 4 'A93' 'A101' 1 'A121' 26 'A14

In [27]:
print(c.error(datos_test, r))

0.91
